# MARATONA BEHIND THE CODE 2020

## DESAFIO 2: UNINASSAU

### Carregando bibliotecas

In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

from sklearn.base import BaseEstimator, TransformerMixin

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Carregando o dataset de treino

In [2]:
# Carregando o dataset da competição
train = pd.read_csv('../Data/train_dataset_uninassau.csv')

### Realizando o pré-processamento dos dados

#### Transformação 1: excluindo colunas do dataset

In [3]:
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        data = X.copy()
        return data.drop(labels=self.columns, axis='columns')

In [4]:
# Instanciando uma transformação DropColumns
rm_columns = DropColumns(
    columns=["NOME","MATRICULA","H_AULA_PRES"]
)

print(rm_columns)

DropColumns(columns=['NOME', 'MATRICULA', 'H_AULA_PRES'])


In [5]:
# Aplicando a transformação ``DropColumns`` ao conjunto de dados base
rm_columns.fit(X=train)

# Reconstruindo um DataFrame Pandas com o resultado da transformação
train2 = pd.DataFrame.from_records(
    data=rm_columns.transform(
        X=train
    ),
)

In [6]:
# Visualizando as colunas do dataset transformado
print("Colunas do dataset após a transformação ``DropColumns``: \n")
print(train2.columns)

Colunas do dataset após a transformação ``DropColumns``: 

Index(['REPROVACOES_DE', 'REPROVACOES_EM', 'REPROVACOES_MF', 'REPROVACOES_GO',
       'NOTA_DE', 'NOTA_EM', 'NOTA_MF', 'NOTA_GO', 'INGLES', 'TAREFAS_ONLINE',
       'FALTAS', 'PERFIL'],
      dtype='object')


#### Transformação 2: tratando dados faltantes

In [7]:
# Criação de um objeto ``SimpleImputer``
si = SimpleImputer(
    missing_values=np.nan,
    strategy='constant',
    fill_value=0,
    verbose=0,
    copy=True
)

In [8]:
# Aplicamos o SimpleImputer ``si`` ao conjunto de dados train2 (resultado da primeira transformação)
si.fit(X=train2)

# Reconstrução de um novo DataFrame Pandas com o conjunto imputado (train3)
train3 = pd.DataFrame.from_records(
    data=si.transform(
        X=train2
    ),
    columns=train2.columns
)

### Treinando um modelo de classificação

#### Definindo as features do modelo

In [9]:
# Definição das colunas que serão features (nota-se que a coluna NOME não está presente)
features = [
    'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
    "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
    "INGLES", "TAREFAS_ONLINE", "FALTAS", 
]

# Definição da variável-alvo
target = ["PERFIL"]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = train3[features]
y = train3[target]

#### Separando o dataset em um conjunto de treino e um conjunto de teste

In [10]:
# Separação dos dados em um conjunto de treino e um conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=337)

<hr>

#### Criando um modelo usando GradientBoosting

In [11]:
# Criação do modelo
gbc = GradientBoostingClassifier(criterion='mse', 
                                 init=None,
                                 learning_rate=0.1, 
                                 loss='deviance', 
                                 max_depth=3,
                                 max_features=None, 
                                 max_leaf_nodes=None,
                                 min_impurity_decrease=0.0, 
                                 min_impurity_split=None,
                                 min_samples_leaf=1, 
                                 min_samples_split=2,
                                 min_weight_fraction_leaf=0.0, 
                                 n_estimators=60,
                                 n_iter_no_change=None, 
                                 presort='auto', 
                                 random_state=None,
                                 subsample=0.5, 
                                 tol=0.0001, 
                                 validation_fraction=0.1,
                                 verbose=0, 
                                 warm_start=False)

#### Testando o classificador

In [12]:
# Treino do modelo (é chamado o método *fit()* com os conjuntos de treino)
gbc.fit(X_train,y_train)

/Users/rodrigolima82/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rodrigolima82/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:1342: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradientBoosting models instead.
  FutureWarning)


GradientBoostingClassifier(ccp_alpha=0.0, criterion='mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=60,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=0.5, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

#### Execução de predições e avaliação do modelo

In [13]:
# Realização de teste cego no modelo criado
y_pred = gbc.predict(X_test)

In [14]:
X_test.head()

,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,INGLES,TAREFAS_ONLINE,FALTAS
5414,0,0,0,0,5.9,5.4,4.6,4.9,1.0,1,6
7475,0,0,1,1,7.1,6.8,0.0,0.0,0.0,6,2
17254,1,1,3,1,0.0,0.0,0.0,0.0,0.0,2,3
1716,0,0,0,0,6.6,6.7,5.1,0.0,1.0,2,6
18538,0,0,0,0,5.8,6.1,5.4,5.0,0.0,2,6


In [15]:
print(y_pred)

['DIFICULDADE' 'DIFICULDADE' 'DIFICULDADE' ... 'HUMANAS' 'EXCELENTE'
 'DIFICULDADE']


In [16]:
# Acurácia alcançada pelo modelo
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred), 2)))

Acurácia: 81.0%


<hr>

## Scoring dos dados necessários para entregar a solução

Como entrega da sua solução, esperamos os resultados classificados no seguinte dataset chamado "to_be_scored.csv":

### Download da "folha de respostas"

In [17]:
df_to_be_scored = pd.read_csv('../Data/to_be_scored_uninassau.csv')

# Atenção!

O dataframe ``to_be_scored`` é a sua "folha de respostas". Note que a coluna "PERFIL" não existe nessa amostra, que não pode ser então utilizada para treino de modelos de aprendizado supervisionado.

In [18]:
df_to_be_scored.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 14 columns):
MATRICULA         500 non-null int64
NOME              500 non-null object
REPROVACOES_DE    500 non-null int64
REPROVACOES_EM    500 non-null int64
REPROVACOES_MF    500 non-null int64
REPROVACOES_GO    500 non-null int64
NOTA_DE           500 non-null float64
NOTA_EM           500 non-null float64
NOTA_MF           500 non-null float64
NOTA_GO           500 non-null float64
INGLES            500 non-null int64
H_AULA_PRES       500 non-null int64
TAREFAS_ONLINE    500 non-null int64
FALTAS            500 non-null int64
dtypes: float64(4), int64(9), object(1)
memory usage: 54.8+ KB


<hr>

# Atenção!

# Para poder aplicar seu modelo e classificar a folha de respostas, você precisa primeiro aplicar as mesmas transformações com colunas que você aplicou no dataset de treino.

# Não remova ou adicione linhas na folha de respostas. 

# Não altere a ordem das linhas na folha de respostas.

# Ao final, as 500 entradas devem estar classificadas, com os valores previstos em uma coluna chamada "target"

<hr>

Na célula abaixo, repetimos rapidamente os mesmos passos de pré-processamento usados no exemplo dado com árvore de decisão

In [19]:
# Aplicando a transformação ``DropColumns`` ao conjunto de dados base
rm_columns.fit(X=df_to_be_scored)

# Reconstruindo um DataFrame Pandas com o resultado da transformação
df_to_be_scored_2 = pd.DataFrame.from_records(
    data=rm_columns.transform(
        X=df_to_be_scored
    ),
)

# Aplicamos o SimpleImputer ``si`` ao conjunto de dados df_data_2 (resultado da primeira transformação)
si.fit(X=df_to_be_scored_2)

# Reconstrução de um novo DataFrame Pandas com o conjunto imputado (df_data_3)
df_to_be_scored_3 = pd.DataFrame.from_records(
    data=si.transform(
        X=df_to_be_scored_2
    ),  # o resultado SimpleImputer.transform(<<pandas dataframe>>) é lista de listas
    columns=df_to_be_scored_2.columns  # as colunas originais devem ser conservadas nessa transformação
)

df_to_be_scored_3.tail()

,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,INGLES,TAREFAS_ONLINE,FALTAS
495,0.0,0.0,0.0,0.0,6.3,5.3,7.0,5.6,1.0,0.0,8.0
496,0.0,0.0,0.0,0.0,5.7,5.9,5.8,5.0,1.0,1.0,6.0
497,0.0,0.0,0.0,0.0,6.7,7.1,5.7,6.3,0.0,6.0,3.0
498,0.0,0.0,0.0,0.0,6.7,7.2,5.2,5.7,1.0,3.0,5.0
499,0.0,0.0,0.0,0.0,7.1,6.8,6.5,6.0,1.0,3.0,6.0


<hr>

Pode ser verificado abaixo que as colunas da folha de resposta agora são idênticas às que foram usadas para treinar o modelo:

In [20]:
df_to_be_scored_3.columns

Index(['REPROVACOES_DE', 'REPROVACOES_EM', 'REPROVACOES_MF', 'REPROVACOES_GO',
       'NOTA_DE', 'NOTA_EM', 'NOTA_MF', 'NOTA_GO', 'INGLES', 'TAREFAS_ONLINE',
       'FALTAS'],
      dtype='object')

### Executando as predições na "folha de respostas"

In [21]:
y_pred = gbc.predict(df_to_be_scored_3)
df_to_be_scored_3['target'] = y_pred
df_to_be_scored_3.tail()

,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,INGLES,TAREFAS_ONLINE,FALTAS,target
495,0.0,0.0,0.0,0.0,6.3,5.3,7.0,5.6,1.0,0.0,8.0,HUMANAS
496,0.0,0.0,0.0,0.0,5.7,5.9,5.8,5.0,1.0,1.0,6.0,EXATAS
497,0.0,0.0,0.0,0.0,6.7,7.1,5.7,6.3,0.0,6.0,3.0,EXATAS
498,0.0,0.0,0.0,0.0,6.7,7.2,5.2,5.7,1.0,3.0,5.0,EXATAS
499,0.0,0.0,0.0,0.0,7.1,6.8,6.5,6.0,1.0,3.0,6.0,EXATAS


### Salvando a folha de respostas como um arquivo .csv para ser submetido

In [22]:
df_to_be_scored_3['target'].value_counts()

EXATAS         215
DIFICULDADE    154
HUMANAS        114
EXCELENTE       12
MUITO_BOM        5
Name: target, dtype: int64

In [23]:
#project.save_data(file_name="results.csv", data=df_to_be_scored_3.to_csv(index=False))
df_to_be_scored_3.to_csv('results.csv',index=False)

<hr>

## Parabéns!

Se você já está satisfeito com a sua solução, vá até a página abaixo e envie os arquivos necessários para submissão.

# https://uninassau.maratona.dev
